In [ ]:
import pysftp
import os 
import numpy as np
import pandas as pd
import netCDF4
import numpy as np

from scipy import io
from classes.matlab_reader import matlab_reader
from classes.grid_funcs import map_grid,dataset,data_wizard
from dotenv import load_dotenv   #for python-dotenv method
from datetime import datetime

In [ ]:
ds = netCDF4.Dataset(file_name)

hour = 3
level = 100

wind_northward = ds['v'][hour][level]# nortward
wind_eastward = ds['u'][hour][level] # eastward
temperature = ds['t'][hour][level]

lat = np.genfromtxt(f'COORDINATES_ERA/COORDINATES_ERA5_LAT.csv', delimiter=',')
lon = np.genfromtxt(f'COORDINATES_ERA/COORDINATES_ERA5_LON.csv', delimiter=',')
grid_ = np.meshgrid(lon,lat)